In [11]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast
import nltk.data

MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256

def split_sentences(text):
    tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
    return tokenizer.tokenize(text)

def correction(seq):
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
    sequences_list = split_sentences(seq)
    task = [f"Spell correct: {sequence}" for sequence in sequences_list]
    tensors = tokenizer(task, padding="longest", max_length=256, truncation=True, return_tensors="pt")
    output = model.generate(**tensors)
    return tokenizer.batch_decode(output, skip_special_tokens=True)  

import difflib, re

def compare_strings(input_, output):
    words_with_errors = []
    punctuation_added = []
    punctuation_remooved = []

    input_arr = re.findall(r"\b\w+\b", input_)
    output_arr = re.findall(r"\b\w+\b", output)

    input_idx = 0
    for word in difflib.ndiff(input_arr, output_arr):
        if word[0] != "+" and word[0] != "?":
            if word[0] == "-":
                words_with_errors.append((input_idx, word[2:]))
            input_idx += 1
    for idx, symbol in enumerate(difflib.ndiff(input_, output)):
        if symbol[0] == "-" and not symbol[2].isalnum() and not symbol[2] == " ":
            punctuation_remooved.append((idx, symbol[2]))
        if symbol[0] == "+" and not symbol[2].isalnum() and not symbol[2] == " ":
            punctuation_added.append((idx, symbol[2]))

    output_args = [', '.join([f'{x[1]}' for x in words_with_errors]), ' | '.join([f'{x[1]}' for x in punctuation_added]),
                   ', '.join([f'{x[0]}' for x in punctuation_added]), ' | '.join([f'{x[1]}' for x in punctuation_remooved]),
                   ', '.join([f'{x[0]}' for x in punctuation_remooved])]

    return (f'Орфографические ошибки в словах: {output_args[0]}.', 
            f'Пропущены знаки: "{output_args[1]}" на позициях под номерами: {output_args[2]}.',
            f'Лишние знаки:  "{output_args[3]}" на позициях под номерами: {output_args[4]}.')

In [12]:
print('Исправленное предложение: ',  correction('Я прешол к тибе с преветом - рассказать что солнца встало')[0], '\n')
for i in compare_strings('Я прешол к тибе с преветом - рассказать что сллнце встало', 
                         correction('Я прешол к тибе с преветом - рассказать что сллнце встало')[0]):
    
    print(i, '\n')

Исправленное предложение:  Я пришел к тебе с приветом - рассказать, что солнце встало. 

Орфографические ошибки в словах: прешол, тибе, преветом, сллнце. 

Пропущены знаки: ", | ." на позициях под номерами: 43, 63. 

Лишние знаки:  "" на позициях под номерами: . 

